In [ ]:
###########import packages##########
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.backend.tensorflow_backend import set_session
from keras import optimizers
from keras import regularizers
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard
from keras.constraints import max_norm
from keras.models import Sequential 
from keras.layers import Dense 
from keras.layers import Dropout 
from keras.models import Model
from keras.layers import BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier 
from keras.wrappers.scikit_learn import KerasRegressor
from keras.constraints import maxnorm 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import GridSearchCV
from sko.GA import GA
from sko.PSO import PSO
import time
from keras.models import Model, load_model
from sko.GA import GA
from sko.DE import DE
from sklearn import tree
%matplotlib
###########assign memory##########
###########delete this part if your tensorflow was based on CPU##########
config = tf.ConfigProto()
config.gpu_options.allocator_type = 'BFC' #A "Best-fit with coalescing" algorithm, simplified from a version of dlmalloc.
config.gpu_options.per_process_gpu_memory_fraction = 0.45
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config)) 
###########fix random seed for reproducability##########
seed=1
np.random.seed(seed)
###########wrapping root mean square error for later calls##########
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())
###########loading data##########
fdata=pd.read_csv('database.csv',encoding="gbk")
raw_data=fdata.loc[:,[                     
                      'A_c',#0
                      'A_a',#1
                      'L_cl',#2
                      'i0',#3
                      'L-PEM',#4
                      'L_GDL',#5
                      'epsl_cl',#6
                      'epsp_cl',#7
                      'T_cc',#8
                      'RH_cc',#9
                      'pressure',#10
                        'Volt0.9',
                        'Volt0.8',
                        'Volt0.7',#13
                        'Volt0.6',
                        'Volt0.5',
                        'Volt0.4',
                        'Volt0.3',
                        'Volt0.2',
                        'Volt0.1',
                        'Volt0'
                        ]]
###########data standardization##########
standardized_data = (raw_data-np.mean(raw_data,axis=0))/np.std(raw_data,axis=0)

###########defining a wrapper function for later call from each machine learning algorithms##########
raw_input=standardized_data.iloc[:,0:11]
raw_output=standardized_data.iloc[:,11:]
X=raw_input.values.astype(np.float32)
y=raw_output.values.astype(np.float32)
###########fix random seed for reproducability##########
seed=1
###########train test splitting##########
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.15,random_state=seed)
raw_input_global=raw_data.iloc[:,0:11]
raw_output_global=raw_data.iloc[:,11:]
###########wrap up fuction for later call for OPTIMIZATION##########
def evaluate(pre_2,real_2):
    pre_2=np.array(pre_2)
    real_2=np.array(real_2)
    pre_2_series=pd.Series(pre_2)
    real_2_series=pd.Series(real_2)
    return rmse(pre_2,real_2), round(pre_2_series.corr(real_2_series), 3)
def compare(list_name,limit):
    judge=1
    for a in list_name:
        if a < limit:
            judge=judge*1
        else:
            judge=judge*0
    return judge
def generate_arrays_from_file(path):
    while True:
        with open(path) as f:
            for line in f:
                # create numpy arrays of input data
                # and labels, from each line in the file
                x1, x2, y = process_line(line)
                yield ({'input_1': x1, 'input_2': x2}, {'output': y})
neurons1=500
regularizer=keras.regularizers.l2(0)
dropout_rate=0
epochs_number=1000
learning_rate_search=0.000005
batch_size_number=32
activation1='relu'
###########keras ANN model construction##########
model = Sequential() 
model.add(Dense(neurons1, input_dim=11, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation=activation1,kernel_regularizer=regularizer)) 
model.add(Dropout(dropout_rate))
model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation=activation1,kernel_regularizer=regularizer)) 
model.add(Dropout(dropout_rate))
model.add(Dense(10, input_dim=neurons1, activation='linear'))
adam=optimizers.Adam(lr=learning_rate_search)
model.compile(loss='mse', optimizer=adam)
print('training...')
model.fit(X_train, y_train,verbose=1, batch_size=batch_size_number,epochs=epochs_number)
result=model.predict(X_test)
###########get RMSE and R2##########
pred_voltage_result_list=np.std(raw_output_global,axis=0).T.values*result+np.mean(raw_output_global,axis=0).T.values
real_voltage_result_list=np.std(raw_output_global,axis=0).T.values*y_test+np.mean(raw_output_global,axis=0).T.values
pred_voltage_result_list_07_series=pd.Series(pred_voltage_result_list[:,2])
real_voltage_result_list_07_series=pd.Series(real_voltage_result_list[:,2])
corr=round(pred_voltage_result_list_07_series.corr(real_voltage_result_list_07_series), 5)
rmse_07=rmse(pred_voltage_result_list[:,2],real_voltage_result_list[:,2])
print('scatter corr',corr,'scatter rmse',rmse_07)
x_y_x=np.arange(0,2.5,0.1)
x_y_y=np.arange(0,2.5,0.1)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(pred_voltage_result_list[:,2],real_voltage_result_list[:,2],color='red')
ax.plot(x_y_x,x_y_y)
plt.legend()
plt.xlabel(u"Predicted_Current_0.7V A")
plt.ylabel(u"Real_Current_0.7V A")
plt.show()


In [ ]:
model.save('ANN_CURVE.h5')

In [ ]:
def plot_scatter(i,space):
    pred_voltage_result_list=np.std(raw_output_global,axis=0).T.values*result+np.mean(raw_output_global,axis=0).T.values
    real_voltage_result_list=np.std(raw_output_global,axis=0).T.values*y_test+np.mean(raw_output_global,axis=0).T.values
    pred_voltage_result_list_07_series=pd.Series(pred_voltage_result_list[:,i])
    real_voltage_result_list_07_series=pd.Series(real_voltage_result_list[:,i])
    corr=round(pred_voltage_result_list_07_series.corr(real_voltage_result_list_07_series), 5)
    rmse_07=rmse(pred_voltage_result_list[:,i],real_voltage_result_list[:,i])
    print('scatter corr',corr,'scatter rmse',rmse_07)
    voltage_now=0.9-0.1*i
    x_y_x=np.arange(0,space,0.1)
    x_y_y=np.arange(0,space,0.1)
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.scatter(pred_voltage_result_list[:,i],real_voltage_result_list[:,i],color='red',label='Artificial Neural Network')
    ax.plot(x_y_x,x_y_y)
    plt.legend()
    plt.xlabel(u"Predicted_Current_%sV A" %voltage_now)
    plt.ylabel(u"Real_Current_%s VA" %voltage_now)
    plt.show()
    plt.savefig('ANN CURVE %sV.png' %voltage_now)
plot_scatter(0,0.75)
plot_scatter(1,1.5)
plot_scatter(2,2.25)
plot_scatter(3,3)
plot_scatter(4,3)


In [ ]:
def processing(thickness):
    median=standardized_data.median()
    raw_median=raw_data.median()
    user_input1=raw_median[0:11]    
    dict_result={}    
    user_input1.iloc[0]=1.872e7
    user_input1.iloc[1]=1.872e7
    user_input1.iloc[2]=thickness
    user_input1.iloc[3]=24.5
    user_input1.iloc[4]=23
    user_input1.iloc[5]=125
    user_input1.iloc[6]=0.19
    user_input1.iloc[7]=0.64
    user_input1.iloc[8]=353.15
    user_input1.iloc[9]=1
    user_input1.iloc[10]=300000
    standardized_user_input1=(user_input1-np.mean(raw_data,axis=0)[0:11])/np.std(raw_data,axis=0)[0:11]
    data_test_input1=standardized_user_input1
    data_test_input1=pd.DataFrame(data_test_input1)
    data_test_input1=data_test_input1.T
    data_test_param1=data_test_input1.values.astype(np.float32)
    predict_ann=model.predict(data_test_param1)
    ###########result output##########
    predict_ann=predict_ann*np.std(raw_output_global,axis=0).T.values+np.mean(raw_output_global,axis=0).T.values
    x_prediction_ann=predict_ann.astype(np.float32)
    x_prediction_ann=x_prediction_ann.tolist()
    x_prediction_ann=x_prediction_ann[0]
#     print(x_prediction_maximum_power_ann)
    x_07=x_prediction_ann[2]
    dict_result.update({x_07:{
                                'thickness':thickness}})

    return x_07, dict_result
def processing_func(p):
    thickness= p
    result, _ = processing(thickness)
    return -result


In [ ]:
def processing_PRINT(thickness):
    median=standardized_data.median()
    raw_median=raw_data.median()
    user_input1=raw_median[0:11]    
    dict_result={}    
    user_input1.iloc[0]=1.872e7
    user_input1.iloc[1]=1.872e7
    user_input1.iloc[2]=thickness
    user_input1.iloc[3]=24.5
    user_input1.iloc[4]=23
    user_input1.iloc[5]=125
    user_input1.iloc[6]=0.19
    user_input1.iloc[7]=0.64
    user_input1.iloc[8]=353.15
    user_input1.iloc[9]=1
    user_input1.iloc[10]=300000
    standardized_user_input1=(user_input1-np.mean(raw_data,axis=0)[0:11])/np.std(raw_data,axis=0)[0:11]
    data_test_input1=standardized_user_input1
    data_test_input1=pd.DataFrame(data_test_input1)
    data_test_input1=data_test_input1.T
    data_test_param1=data_test_input1.values.astype(np.float32)
    predict_ann=model.predict(data_test_param1)
    ###########result output##########
    predict_ann=predict_ann*np.std(raw_output_global,axis=0).T.values+np.mean(raw_output_global,axis=0).T.values
    x_prediction_ann=predict_ann.astype(np.float32)
    x_prediction_ann=x_prediction_ann.tolist()
    x_prediction_ann=x_prediction_ann[0]
#     print(x_prediction_maximum_power_ann)
    x_07=x_prediction_ann[2]
    dict_result.update({x_07:{
                                'thickness':thickness}})
    print(x_07)

    return x_07, dict_result
import time
time_start = time.time()
for i in np.arange(1,15,0.01):
    processing_PRINT(i)

time_end = time.time()
print('Time cost of grid search= %fs' % (time_end - time_start))

In [ ]:
%matplotlib
iter_num = 32
ga = GA(func=processing_func, n_dim=1, size_pop=50, max_iter=iter_num, lb=[1], ub=[10],precision=1e-2)
import time
time_start = time.time()
best_x, best_y = ga.run()
print('best_x:', best_x, '\n', 'best_y:', best_y)
time_end = time.time()
print('Time cost of grid search= %fs' % (time_end - time_start))

# visualization 
import pandas as pd
import matplotlib.pyplot as plt

Y_history = pd.DataFrame(ga.all_history_Y)
fig, ax = plt.subplots(2, 1, figsize=(6, 9))
ax[0].plot(Y_history.index, Y_history.values, '.', color='red')
Y_history.min(axis=1).cummin().plot(kind='line')
plt.show()
plt.savefig('GA OPT.png')

In [ ]:
%matplotlib
def processing_curve(thickness):
    median=standardized_data.median()
    raw_median=raw_data.median()
    user_input1=raw_median[0:11]    
    dict_result={}    
    user_input1.iloc[0]=1.872e7
    user_input1.iloc[1]=1.872e7
    user_input1.iloc[2]=thickness
    user_input1.iloc[3]=24.5
    user_input1.iloc[4]=23
    user_input1.iloc[5]=125
    user_input1.iloc[6]=0.19
    user_input1.iloc[7]=0.64
    user_input1.iloc[8]=353.15
    user_input1.iloc[9]=1
    user_input1.iloc[10]=300000
    standardized_user_input1=(user_input1-np.mean(raw_data,axis=0)[0:11])/np.std(raw_data,axis=0)[0:11]
    data_test_input1=standardized_user_input1
    data_test_input1=pd.DataFrame(data_test_input1)
    data_test_input1=data_test_input1.T
    data_test_param1=data_test_input1.values.astype(np.float32)
    predict_ann=model.predict(data_test_param1)
    ###########result output##########
    predict_ann=predict_ann*np.std(raw_output_global,axis=0).T.values+np.mean(raw_output_global,axis=0).T.values
    x_prediction_ann=predict_ann.astype(np.float32)
    x_prediction_ann=x_prediction_ann.tolist()
    x_prediction_ann=x_prediction_ann[0]
    return x_prediction_ann[0:]
def get_power_density(list1,list2):
    result_list=[]
    for i in range(0,len(list1)):
        result=list1[i]*list2[i]
        result_list.append(result)
#     print(result_list)
    return result_list

thickness_list=[2.5,4,5,7.5,10,15]
voltage_list=[0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1,0]
print(processing_curve(2.5))
print(get_power_density(processing_curve(2.5),voltage_list))
print(processing_curve(4))
print(get_power_density(processing_curve(4),voltage_list))
print(processing_curve(5))
print(get_power_density(processing_curve(5),voltage_list))
print(processing_curve(7.5))
print(get_power_density(processing_curve(7.5),voltage_list))
print(processing_curve(10))
print(get_power_density(processing_curve(10),voltage_list))
print(processing_curve(15))
print(get_power_density(processing_curve(15),voltage_list))
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(processing_curve(2.5),voltage_list,color='blue')
ax.plot(processing_curve(4),voltage_list,color='red')
ax.plot(processing_curve(5),voltage_list,color='green')
ax.plot(processing_curve(7.5),voltage_list,color='black')
ax.plot(processing_curve(10),voltage_list,color='yellow')
ax.plot(processing_curve(15),voltage_list,color='orange')
plt.show()

In [ ]:
import shap
# %matplotlib
standardized_data_SHAP = standardized_data.sample(5000,random_state=1)
SHAP_INPUT=standardized_data_SHAP.iloc[:,0:11]
SHAP_OUTPUT=standardized_data_SHAP.iloc[:,13]
X_SHAP=SHAP_INPUT.values.astype(np.float32)
y_SHAP=SHAP_OUTPUT.values.astype(np.float32)
explainer = shap.DeepExplainer(model,X_SHAP)
cols=[ 
              'A_c',#0
                      'A_a',#1
                      'L_cl',#2
                      'i0',#3
                      'L-PEM',#4
                      'L_GDL',#5
                      'epsl_cl',#6
                      'epsp_cl',#7
                      'T_cc',#8
                      'RH_cc',#9
                      'pressure'
                        ]
shap_values = explainer.shap_values(X_SHAP)
print(type(shap_values))

In [ ]:
shap.summary_plot(shap_values[2], SHAP_INPUT)